In [ ]:
from langchain_community.chat_models import ChatZhipuAI
from dotenv import load_dotenv
from utilis import *
import os

load_dotenv()
llm_key = os.getenv('ZHIPUAI_API_KEY')
db_key = os.getenv('DB_KEY')

llm = ChatZhipuAI(
    temperature=0.1,
    api_key=llm_key,
    model_name="glm-4-flash",
)

connection = create_db_connection("localhost", "root", db_key, "用户画像")

In [73]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate

template_sql = (
    "数据库信息如下：\n"
    "{info}\n"
    "注释信息如下：\n"
    "{comments}\n"
    "需要回答的问题是：\n"
    "{question}\n"
    "如果需要回答的问题跟数据库毫无关系，则不要回答\n"
    "请通过写 MYSQL代码来回答问题。请确保你的代码不要使用 PostgreSQL特有的语法而是选择 MySQL语法\n"
    "在回答问题之前，请先查看注释信息，了解所有列的定义以及数字类别的含义。\n"
    "注意你需要通过 sql代码回答，不需要文字。\n"
    "代码格式如下：```sql\n"
    "..."
)
t_sql=PromptTemplate.from_template(template_sql)


schema_info = get_schema(connection)
comments = get_comments(connection)

chain_sql=({"info":RunnablePassthrough(),         
            "question":RunnablePassthrough(),"comments":RunnablePassthrough()}
           |t_sql
           |llm
           |StrOutputParser()
           |RunnableLambda(get_sql))

In [74]:
template_nlp = (
    "请通过综合如下的数据库信息，sql代码的执行结果给出问题的自然语言回答。\n"
    "数据库信息{info}\n"
    "需要回答的问题是：{question}\n"
    "sql代码: {query}\n"
    "sql代码执行结果: {res}\n"
)

t_ans=PromptTemplate.from_template(template_nlp)

chain_nlp=({"info":RunnablePassthrough(),
            "question":RunnablePassthrough(),
            "query":chain_sql}
            |RunnablePassthrough.assign(res=lambda x: run_query(connection,x["query"]))
            |t_ans
            |llm
            |StrOutputParser())

In [ ]:
#得到SQL代码

question = ""
input_data = {"info": schema_info, "question": question, "commetns": comments}

# chain_sql.invoke(input_data)
for chunk in chain_sql.stream(input_data):
    print(chunk, end="", flush=True)

In [ ]:
#得到NLP回答

# result = chain_ans.invoke(input_data)
# print(result)

for chunk in chain_nlp.stream(input_data):
    print(chunk, end="", flush=True)